In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from urllib.request import urlopen
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**IMPORTING LIBRARIES**

In [7]:

import pandas as pd
import numpy as np 
from bs4 import BeautifulSoup
import requests
import json
import urllib.request as request
import re
import datetime
import time 
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import ast

In [2]:
from bs4 import BeautifulSoup

In [3]:
!pip install requests
!pip install html5lib

In [4]:
import requests
URL = "https://store.steampowered.com/explore/new/"
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html5lib') # If this line causes an error, run 'pip install html5lib' or install html5lib
print(soup.prettify())

<!DOCTYPE html>
<html class=" responsive" lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <meta content="#171a21" name="theme-color"/>
  <title>
   New On Steam
  </title>
  <link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="https://store.cloudflare.steamstatic.com/public/shared/css/motiva_sans.css?v=2C1Oh9QFVTyK&amp;l=english&amp;_cdn=cloudflare" rel="stylesheet" type="text/css"/>
  <link href="https://store.cloudflare.steamstatic.com/public/shared/css/shared_global.css?v=fNZ9ezzB4gzo&amp;l=english&amp;_cdn=cloudflare" rel="stylesheet" type="text/css"/>
  <link href="https://store.cloudflare.steamstatic.com/public/shared/css/buttons.css?v=hFJKQ6HV7IKT&amp;l=english&amp;_cdn=cloudflare" rel="stylesheet" type="text/css"/>
  <link href="https://store.cloudflare.steamstatic.com/public/css/v6/store.css?v=ClCf5lxeGCep&amp;l=english&amp;_cdn=cloudf

EXTRACTING LINKS WHICH CONSIST OF GAMES ONLY

In [5]:
link = []
for links in soup.find_all("a"):
    if links!=None:
        link.append(links.get('href'))
conv = lambda i : i or ''
res = [conv(i) for i in link]
res

['https://store.steampowered.com/login/?redir=explore%2Fnew%2F&redir_ssl=1&snr=1_239_new__global-header',
 'https://store.steampowered.com/?snr=1_239_new__global-responsive-menu',
 'https://store.steampowered.com/?snr=1_239_new__global-responsive-menu',
 'https://store.steampowered.com/explore/?snr=1_239_new__global-responsive-menu',
 'https://steamcommunity.com/my/wishlist/',
 'https://store.steampowered.com/points/shop/?snr=1_239_new__global-responsive-menu',
 'https://store.steampowered.com/news/?snr=1_239_new__global-responsive-menu',
 'https://store.steampowered.com/stats/?snr=1_239_new__global-responsive-menu',
 'https://steamcommunity.com/',
 'https://steamcommunity.com/',
 'https://steamcommunity.com/discussions/',
 'https://steamcommunity.com/workshop/',
 'https://steamcommunity.com/market/',
 'https://steamcommunity.com/?subsection=broadcasts',
 'https://help.steampowered.com/en/',
 'https://store.steampowered.com/privacy_agreement/?snr=1_239_new__global-responsive-menu',
 'h

In [6]:
proper_links = []
for i in res: 
    if(i.startswith('https://store.steampowered.com/app/')):
        proper_links.append(i)
    else:
        pass
proper_links

['https://store.steampowered.com/app/454120/Starbase/?snr=1_239_new__newonsteam-largecap',
 'https://store.steampowered.com/app/1123050/GRIME/?snr=1_239_new__newonsteam-largecap',
 'https://store.steampowered.com/app/979690/The_Ascent/?snr=1_239_new__newonsteam-newtopsellers',
 'https://store.steampowered.com/app/858820/Tribes_of_Midgard/?snr=1_239_new__newonsteam-newtopsellers',
 'https://store.steampowered.com/app/1123050/GRIME/?snr=1_239_new__newonsteam-newtopsellers',
 'https://store.steampowered.com/app/1127500/Mini_Motorways/?snr=1_239_new__newonsteam-newtopsellers',
 'https://store.steampowered.com/app/1016800/Chernobylite/?snr=1_239_new__newonsteam-newtopsellers',
 'https://store.steampowered.com/app/454120/Starbase/?snr=1_239_new__newonsteam-newtopsellers',
 'https://store.steampowered.com/app/1522820/Orcs_Must_Die_3/?snr=1_239_new__newonsteam-newtopsellers',
 'https://store.steampowered.com/app/686810/Hell_Let_Loose/?snr=1_239_new__newonsteam-newtopsellers',
 'https://store.s

## **SCRAPING THE DATA**
Data is scraped using BeautifulSoup by parsing html.The data is uncleaned and not organised after extracting the data.

In [8]:
description = []
developer = []
publisher = []
review_count = []
title = []
review_status = []
count = 0
review = []
#System Configuration....................................................................
for i in range(0,6):
    url = proper_links[i]
    r=requests.get(url)
    soup = BeautifulSoup(r.content,'html.parser')
    for i in soup.find_all(attrs = {'class':'sysreq_contents'}):
        al = i.find('ul')
        description.append(al.text)
#titles....................................................................................
    
    for i in soup.find_all(attrs = {'class':"apphub_AppName"}):
        title.append(i.get_text())
    #titles = soup.find('title')
    #title.append(titles)



#Developer & Publisher....................................................................
    
    
    for i in soup.find_all(attrs = {'class':'dev_row'}):
      #for j in i.find_all(attrs = {'class':'subtitle column'}):
      name=i.find('a')
      #d.append(name)
      developer.append(i.text)
    #for j in soup.find_all(attrs = {'class':'game_area_sys_req_full'}):
      #e.append(j.text)

#Review Count............................................................................
    for i in soup.findAll("span", ["class","responsive_hidden"]):
        #for j in i.findAll(attrs={'class':'user_reviews_count'}):
        review_count.append(i.text)

In [9]:
review_count

['\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t(2,204)\r\n\t\t\t\t\t\t\t\t\t\t\t\t',
 ':',
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t(212)\r\n\t\t\t\t\t\t\t\t\t\t\t\t',
 ':',
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t(3,754)\r\n\t\t\t\t\t\t\t\t\t\t\t\t',
 ':',
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t(4,173)\r\n\t\t\t\t\t\t\t\t\t\t\t\t',
 ':',
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t(212)\r\n\t\t\t\t\t\t\t\t\t\t\t\t',
 ':',
 '\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t(3,775)\r\n\t\t\t\t\t\t\t\t\t\t\t\t',
 ':']

### **CLEANING THE DATA**
Data  is cleaned and organised and stored in DataFrame.

**TITLES**

In [10]:
titles = []
for i in title:
    if(i not in titles):
        titles.append(i)
titles

['Starbase', 'GRIME', 'The Ascent', 'Tribes of Midgard', 'Mini Motorways']

**Number Of reviews**

In [11]:
num= []
k = []
for i in review_count:
    i = i.split()
    for j in i:
        num.append(i)
for i in num:
    for j in i:
        k.append(j)

a = ""
for i in k:
    a+=i

    
    
b = ""
for i in a:
    b += i
b = b.replace(',','')
b = b.replace(':','')
b = b.replace('(','')
res = [int(i) for i in b.split(')') if i.isdigit()]
del res[0]
res

[212, 3754, 4173, 212, 3775]

**Specification**

After extracting the minimum specification, I cleaned and processed data by converting a list into dataframe and eliminating the tags and converting each columns into list. 

In [14]:
a = ""
for i in description:
    a+=i
df = pd.DataFrame([x.split(':') for x in a.split('\n')])
df = df.drop(index=df.index[0],axis=0)
df = df.drop(df.columns[0], axis=1)
df = df.rename(columns={1:'Minimum',2:'OS',3:'Processor',4:'Memory',5:'Graphics',6:'Storage',7:'Internet'})
df['Minimum'] = np.where(df['Minimum']=='OS','N/A',df['Minimum'])
df['OS'] = df.OS.str.rsplit(' ',1).str[0]
df['Processor'] = df.Processor.str.rsplit(' ',1).str[0]
df['Memory'] = df.Memory.str.rsplit(' ',1).str[0]
df['Graphics'] = df.Graphics.str.rsplit(' ',1).str[0]
df['Storage'] = df.Storage.str.rsplit(' ',1).str[0]
df = df.drop(['Internet'], axis = 1)
df = df.drop([8], axis = 1)
df = df.drop([9], axis = 1)
#df = df.drop([10],axis=1)
df['Graphics'] = df['Graphics'].replace('','N/A')
df['Storage'] = df['Storage'].replace('','N/A')
df = df.drop(index=df.index[5],axis=0)
df.head()
minimum = df['Minimum'].tolist()
OS = df['OS'].tolist()
processor = df['Processor'].tolist()
memory = df['Memory'].tolist()
graphics = df['Graphics'].tolist()
storage = df['Storage'].tolist()

# # # **PIPELINE**

After extracting all the specified information, I combined the required columns into list and made a dataframe by combining all the column list.

In [15]:
update = []
ab = []
for i in developer:
    update.append(i.replace("\n", ""))
for i in update:
    ab.append((i.split(':')))
df = pd.DataFrame(ab)
df['Developer'] = np.nan
df['Publisher'] = np.nan
df['Developer'] = np.where(df[0]=='Developer',df[1],np.nan)
df['Publisher'] = np.where(df[0]=='Publisher',df[1],np.nan)
df['Franchise'] = np.where(df[0]=='Franchise',df[1],0)
df = df.drop([0, 1], axis = 1)
df['dev'] = df['Developer'].astype(str) + df['Publisher'].astype(str)
df = df.drop(['Franchise'],axis=1)
df = df.drop(['dev'],axis=1)
df['Developer'] = df['Developer'].fillna(method='ffill').astype(str)
df['Publisher'] = df['Publisher'].fillna(method='bfill').astype(str)
a = []
j = []
unique_developers = []
unique_publisher = []
for i in df['Developer']:
    a.append(i.replace(" ",""))

for i in a:
    if i not in unique_developers:
        unique_developers.append(i)

for i in df['Publisher']:
    j.append(i.replace(" ",""))

for i in j:
    if i not in unique_publisher:
        unique_publisher.append(i)
d = {'Title':titles,'Publisher':unique_publisher,'Developers':unique_developers,'Number of reviews':res,'Minimum':minimum,'OS':OS,'Processor':processor,'Memory':memory,'Graphics':graphics,'Storage':storage}
df = pd.DataFrame(d)
df.head()

,Title,Publisher,Developers,Number of reviews,Minimum,OS,Processor,Memory,Graphics,Storage
0,Starbase,Frozenbyte,Frozenbyte,212,Requires a 64-bit processor and operating syst...,Windows 10,AMD Ryzen 5 / Intel,16 GB,NVIDIA GeForce GTX 1060 6 GB / AMD Radeon RX ...,Version
1,GRIME,AkuparaGames,CloverBite,3754,N/A,Windows,Intel i5 3470 or AMD,4 GB,nVidia GeForce 960 or AMD,Version
2,The Ascent,CurveDigital,NeonGiant,4173,Requires a 64-bit processor and operating syst...,Windows 10,Intel Core Intel Core i5-3470 (4 * 3200) or e...,8 GB,GeForce GTX 660 ( 2048 MB) / Radeon R9 390X (...,35 GB available spaceAdditional
3,Tribes of Midgard,GearboxPublishing,Norsfell,212,N/A,Windows 7 64,Intel Quad Core i5-2300 or AMD,8 GB,Nvidia GeForce GTX 560 (1GB) or AMD Radeon HD...,Version
4,Mini Motorways,DinosaurPoloClub,DinosaurPoloClub,3775,N/A,Windows,Intel i5 3470 or AMD,4 GB,nVidia GeForce 960 or AMD,Version


# **REVIEWS**

To do this,I extracted the URL from inspect element under view all review summary.I then, used that url to get all reviews for a particular game title.Since the requirement is to extract 10 reviews,I cleaned the data removing html tags and slicing the list to view the reviews.

###**STARBASE**

Taking 10 reviews from game title starbase along with date of posting.

In [16]:
url = "https://steamcommunity.com/app/454120/reviews/?browsefilter=toprated&snr=1_5_100010_"
try:
    page = urlopen(url)
except:
    print("Error opening the URL")
soup = BeautifulSoup(page, 'html.parser')
starbase = []
for i in range(0,10):
    for j in soup.find_all("div",class_='apphub_CardTextContent'):
    #for i in j.findAll('div',{'class':'content'}):
        starbase.append(j.text)
a = []
for i in starbase:
    a.append(i.split('Posted:'))
b = a[:11]

k = []
for i in b:
    for j in i:
        k.append(j.replace('\r',''))
l = []
for i in k:
    l.append(i.replace('\t',''))
    
m = []
for i in l:

    m.append(i.split('\n'))
reviews_starbase = []
for i in range(0, len(m)):
    if i % 2:
        reviews_starbase .append(m[i])
reviews_starbase

[[' July 29',
  'Early Access Review',
  'The amount of angry reactive negative reviews is out of hand down here. The store page clearly states this game is missing features and has plenty of bugs as it is in and alpha state.TL;DR           Game has problems mostly from minor teething issues turning into full blown growing pains on launch. Devs know about them and are actively working to fix. I\'d recommend you stay away for the time being unless you\'re desperate to be playing this right now.For what it\'s worth, I\'m enjoying my time with the game despite the issues."Building is broken, and that ruins the game."           I agree most with this, and it\'s a large part of why I\'m negatively reviewing. However, the developers know and are working on it. I\'m gonna sound like a broken record after this: This is an Early Access game. They are not meant to be 100% polished and finished products."I can\'t refuel my ship" is an issue because the tutorials barely cover anything.           Y

### # **MINI-MOTORWAYS**

In [17]:
url = "https://steamcommunity.com/app/1127500/reviews/?browsefilter=toprated&snr=1_5_100010_"
try:
    page = urlopen(url)
except:
    print("Error opening the URL")
soup = BeautifulSoup(page, 'html.parser')
mini_motorways = []
for i in range(0,10):
    for j in soup.find_all("div",class_='apphub_CardTextContent'):
    #for i in j.findAll('div',{'class':'content'}):
        mini_motorways.append(j.text)

a = []
for i in mini_motorways:
    a.append(i.split('Posted:'))
b = a[:11]

k = []
for i in b:
    for j in i:
        k.append(j.replace('\r',''))
l = []
for i in k:
    l.append(i.replace('\t',''))
    
m = []
for i in l:

    m.append(i.split('\n'))
reviews_mini_motorways = []
for i in range(0, len(m)):
    if i % 2:
        reviews_mini_motorways .append(m[i])
reviews_mini_motorways

[[' July 22',
  "Need a creative mode (disable achievement) so you can move business and stop spawning. I just want to watch the city run. I'm here to relax, not to stress out"],
 [' July 25', 'draw roadwatch cards arrive at destinationendorphins'],
 [' July 20',
  "I'm not very far into the game yet but so far gameplay-wise it's pretty much what you'd expect from the devs of mini metro and the trailers and screenshots. soothing music, simple clean and pretty aesthetics, and obviously pretty fun to play.  the reason my lazy ass is even bothering to write a review is because not only does the game _have_ colorblind mode, but when I asked about it on the forums I got a response from the devs and then they followed up that comment by deliberately showing off colorblind mode in their gameplay stream. turns out for me personally it wasn't even much of a deal but the fact that they care enough to add a colorblind mode and then show it off is worthy of my purchase and respect"],
 [' July 25',

# # **F1 2021**

In [18]:
url = 'https://steamcommunity.com/app/1134570/reviews/?browsefilter=toprated&snr=1_5_100010_'
try:
    page = urlopen(url)
except:
    print("Error opening the URL")
soup = BeautifulSoup(page, 'html.parser')
f1 = []

for i in range(0,10):
    for j in soup.find_all("div",class_='apphub_CardTextContent'):
    #for i in j.findAll('div',{'class':'content'}):
        f1.append(j.text)

a = []
for i in f1:
    a.append(i.split('Posted:'))
b = a[:11]

k = []
for i in b:
    for j in i:
        k.append(j.replace('\r',''))
l = []
for i in k:
    l.append(i.replace('\t',''))
    
m = []
for i in l:

    m.append(i.split('\n'))
reviews_f1 = []
for i in range(0, len(m)):
    if i % 2:
        reviews_f1 .append(m[i])
reviews_f1

[[' July 19',
  'Very realistic, my first race on the game and Mazepin span off on the first lap. 10/10 would recommend'],
 [' July 18',
  'Game is super realistic, got ahead of Hamilton and he pitted me into a wall going 180mph, just like real life!'],
 [' July 15', 'Helps me forget about her'],
 [' July 19',
  "I've played F1 2020 before and F1 2021's AI and Damage Model is miles greater than F1 2020. In F1 2020, the AI doesn't try to avoid you and their cars feel heavier than yours. In F1 2021, the AI is more aggressive and defensive with overtaking and actively avoids collisions. I'm playing on an Xbox controller with all assists off and I've been having loads of fun."],
 [' July 19',
  'The realism in this game is insane. Hamilton came down the inside of me and took me out the race. He then preceded to win the race and run around like crazy and even stopped to do the L dance on my destroyed car'],
 [' July 17', 'Mazepin is a bit too fast'],
 [' July 17',
  'To make this as short a

# # **ASCENT**

In [19]:
url = 'https://steamcommunity.com/app/979690/reviews/?browsefilter=toprated&snr=1_5_100010_'
try:
    page = urlopen(url)
except:
    print("Error opening the URL")
soup = BeautifulSoup(page, 'html.parser')
ascent = []

for i in range(0,10):
    for j in soup.find_all("div",class_='apphub_CardTextContent'):
    #for i in j.findAll('div',{'class':'content'}):
        ascent.append(j.text)
a = []
for i in ascent:
    a.append(i.split('Posted:'))
b = a[:11]

k = []
for i in b:
    for j in i:
        k.append(j.replace('\r',''))
l = []
for i in k:
    l.append(i.replace('\t',''))
    
m = []
for i in l:

    m.append(i.split('\n'))
reviews_ascent = []
for i in range(0, len(m)):
    if i % 2:
        reviews_ascent .append(m[i])
reviews_ascent

[[' July 29',
  "I can't distinguish the enemies from the civilians in a firefight.I've killed hundreds of innocent people.Nobody cares."],
 [' July 29',
  'Launched the game, was really hoping for some 80s bladerunner style synths and music... was not disappointed.  Graphics are amazing, Gameplay is smooth as hell and more complex then other games of this style.  Runs super smooth and the gun play is fantastic.  In a world where games come out half finished and unpolished, its nice to play something that actually works as it should and was released when it was supposed to. I really recommend trying this game if you want to have an atmospheric feel like games used to back in the 90s and early 2000s .'],
 [' July 31',
  'Really enjoying this so far. I keep seeing reviews online describing this as if its "doing what cyberpunk 2077 didn\'t do"Don\'t be bullshitted. This isn\'t some massive in depth RPG and the only comparison to cyberpunk 2077 is a loose similarity in aesthetic and the ab

# **TRIBES OF MIDGARD**

In [20]:
url = 'https://steamcommunity.com/app/858820/reviews/?browsefilter=toprated&snr=1_5_100010_'
try:
    page = urlopen(url)
except:
    print("Error opening the URL")
soup = BeautifulSoup(page, 'html.parser')
tribes_of_midgard = []

for i in range(0,10):
    for j in soup.find_all("div",class_='apphub_CardTextContent'):
    #for i in j.findAll('div',{'class':'content'}):
        tribes_of_midgard.append(j.text)
a = []
for i in tribes_of_midgard:
    a.append(i.split('Posted:'))
b = a[:11]

k = []
for i in b:
    for j in i:
        k.append(j.replace('\r',''))
l = []
for i in k:
    l.append(i.replace('\t',''))
    
m = []
for i in l:

    m.append(i.split('\n'))
reviews_tribes_of_midgard = []
for i in range(0, len(m)):
    if i % 2:
        reviews_tribes_of_midgard.append(m[i])
reviews_tribes_of_midgard

[[' July 27',
  "Well, judging from the Steam description, they really marketed the game poorly. It's not an action-survival RPG, but a rogue-lite with open world and RPG elements. The gameplay cycle is to gather, craft better gear, loot bases, etc to defend your base daily from monsters. If you can't keep up that pace or are too far away before the monsters come, your base will be destroyed and its game over. You get permanent improvements for your character after your run. If they marketed this correctly, it would have been considered a pretty good rogue-lite. Instead they called it an action-survival RPG and all the people wanting the next Diablo are upset."],
 [' July 27',
  'Decent game, if you have friends. Terrible if you are a loser with no friends, like me.'],
 [' July 31',
  'If you\'re expecting something like Valheim; stop. Too many times the comparison gets drawn out because "muh vikings craft!" but they couldn\'t be further apart.Tribes of Midgard is a roguelite game wher